# 平均報酬におけるUCRL2アルゴリズム

参考：
* [Near-optimal Regret Bounds for Reinforcement Learning](https://www.jmlr.org/papers/volume11/jaksch10a/jaksch10a.pdf)
* [Bandit Algorithms](https://tor-lattimore.com/downloads/book/book.pdf)の38章

今回は平均報酬でMinimax regretを達成するアルゴリズムを学びます．

表記：

* MDP：$M=(\mathcal{S}, \mathcal{A}, P, r, \mu)$
* 平均報酬（ゲイン）：$\rho_s^\pi=\lim _{n \rightarrow \infty} \frac{1}{n} \sum_{t=1}^n \mathbb{E}^\pi\left[r_{A_t}\left(S_t\right) \mid S_1=s\right]$,
    * この平均は存在するとは限りません．なので，次のlimsupを考える場合があります．これは必ず存在します．$\bar{\rho}_s^\pi=\limsup _{n \rightarrow \infty} \frac{1}{n} \sum_{t=1}^n \mathbb{E}^\pi\left[r_{A_t}\left(S_t\right) \mid S_1=s\right]$
    * 最適ゲイン：$\rho^*=\max _{s \in \mathcal{S}} \sup _\pi \bar{\rho}_s^\pi$
    * Communicatingではない場合，最適ゲインが定数ではない場合があります．
    * メモリーレス方策$\pi$のゲインは次を満たします．$\rho^\pi=\lim _{n \rightarrow \infty} \frac{1}{n} \sum_{t=1}^n P_\pi^{t-1} r_\pi=P_\pi^* r_\pi$．ここで，$P_\pi^*=\lim _{n \rightarrow \infty} \frac{1}{n} \sum_{t=1}^n P_\pi^{t-1}$は定常遷移行列です．
* $v_\pi^{(k)}=\sum_{t=1}^k P_\pi^{t-1}\left(r_\pi-\rho^\pi\right)$を，$k$ステップ目までの過剰報酬とします．
    * （差分）価値関数は$v_\pi=\lim _{n \rightarrow \infty} \frac{1}{n} \sum_{k=1}^n v_\pi^{(k)}=\left(\left(I-P_\pi+P_\pi^*\right)^{-1}-P_\pi^*\right) r_\pi$で定義されます．
    * ちなみに任意のメモリーレス方策について，$\rho^\pi+v_\pi=r_\pi+P_\pi v_\pi$が成立します．
* スパン：$\operatorname{span}(v)=\max _{s \in \mathcal{S}} v(s)-\min _{s \in \mathcal{S}} v(s)$
* MDPの半径：$D(M)=\max _{s \neq s^{\prime}} \min _{\pi \in \Pi_{\mathrm{DM}}} \mathbb{E}^\pi\left[\min \left\{t \geq 1: S_t=s\right\} \mid S_1=s^{\prime}\right]-1$
    * 有限な半径のMDPを特にCommunicatingと呼びます．これはCommunicatingであることの必要十分条件です．（Exercise参照）
    * このとき，最適な平均報酬は状態に依存しません：$\rho^*(M):=\rho^*(M, s):=\max _\pi \rho(M, \pi, s)$.
    * 半径は必ず$\log _A(S)-3$以上になります．

### ベルマン方程式について

* 貪欲方策：$\pi_v(s)=\operatorname{argmax}_{a \in \mathcal{A}} r_a(s)+\left\langle P_a(s), v\right\rangle$
* ベルマン最適方程式：$\rho+v(s)=\max _{a \in \mathcal{A}}\left(r_a(s)+\left\langle P_a(s), v\right\rangle\right) \quad$ for all $s \in \mathcal{S}$

次が成立します．

---

* $(\rho, v)$がベルマン最適方程式を満たす場合，$\rho=\rho^*$ かつ $\pi_v$が最適方策になります．
* 決定的かつメモリーレスな最適方策が存在します．

**証明**

$(\rho, v)$をベルマン最適方程式を満たすとします．また，$\pi^*=\pi_v$を$v$についての貪欲方策とします．
このとき，

$$
\rho^{\pi^*}=\lim _{n \rightarrow \infty} \frac{1}{n} \sum_{t=1}^n P_{\pi^*}^{t-1} r_{\pi^*}=\lim _{n \rightarrow \infty} \frac{1}{n} \sum_{t=1}^n P_{\pi^*}^{t-1}\left(\rho \mathbf{1}+v-P_{\pi^*} v\right)=\rho \mathbf{1} .
$$



---


## UCRL2アルゴリズム

![UCRL2](figs/UCRL2.png)

UCRL2アルゴリズムではExtended value iterationアルゴリズムを走らせるので，先にこちらを紹介します．

### Extended Value Iterationアルゴリズム

* $u_i(s)$を$i$イテレーション目の状態の価値とします．
* $u'_i(s)$をnormalizeされた価値とします

Extended value iterationは次を繰り返します．

$$
\begin{aligned}
u_0(s) & =0, \\
u_{i+1}(s) & =\max _{a \in \mathcal{A}}\left\{\tilde{r}_k(s, a)+\max _{p(\cdot) \in \mathcal{P}(s, a)}\left\{\sum_{s^{\prime} \in \mathcal{S}} p\left(s^{\prime}\right) \cdot u_i\left(s^{\prime}\right)\right\}\right\}
\end{aligned}
$$

ここで，$\tilde{r}_k$は下の集合の内最大のもの，$\mathcal{P}$は下の集合そのものを表します．
$$
\begin{aligned}
\left|\tilde{r}(s, a)-\hat{r}_k(s, a)\right| & \leq \sqrt{\frac{7 \log \left(2 S A t_k / \delta\right)}{2 \max \left\{1, N_k(s, a)\right\}}} \\
\left\|\tilde{p}(\cdot \mid s, a)-\hat{p}_k(\cdot \mid s, a)\right\|_1 & \leq \sqrt{\frac{14 S \log \left(2 A t_k / \delta\right)}{\max \left\{1, N_k(s, a)\right\}}} .
\end{aligned}
$$

$\max_p$は線形計画法で解くことができます．
この価値反復法は
$$
\max _{s \in S}\left\{u_{i+1}(s)-u_i(s)\right\}-\min _{s \in S}\left\{u_{i+1}(s)-u_i(s)\right\}<\frac{1}{\sqrt{t_k}}
$$
で停止します．

### Extended Value Iterationの性質

---

次が成立します：

$$
\max _s u_i(s)-\min _s u_i(s) \leq D
$$

**証明**

まず，$u_i(s)$は$\tilde{M}^+$において，状態$s$から始まる累積期待報酬和であることを思い出しましょう．


## UCRL2の解析

UCRL2のリグレットを導出します．
$k$エピソード目のリグレットを

$$
\Delta_k:=\sum_{s, a} \nu_k(s, a)\left(\rho^*-\bar{r}(s, a)\right)
$$

として定義しましょう．ここで，$\nu_k$はエピソード内で$(s, a)$を訪れた回数です．

---

このとき，次が高確率で成立します．

$$
\sum_k \Delta_k+\sqrt{\frac{5}{2} T \log \left(\frac{8 T}{\delta}\right)}
$$

**証明**

Hoeffdingの不等式から，$T$ステップ間の状態行動のカウント$N(s, a)$に対して，
$$
\mathbb{P}\left\{\left.\sum_{t=1}^T r_t \leq \sum_{s, a} N(s, a) \bar{r}(s, a)-\sqrt{\frac{5}{8} T \log \left(\frac{8 T}{\delta}\right)} \right\rvert\,(N(s, a))_{s, a}\right\} \leq\left(\frac{\delta}{8 T}\right)^{5 / 4}<\frac{\delta}{12 T^{5 / 4}}
$$

が成立します（5は多分$T^{5/4}$でlog取ったら出てくる）．
よって，
$$
\Delta\left(s_1, T\right)=T \rho^*-\sum_{t=1}^T r_t<T \rho^*-\sum_{s, a} N(s, a) \bar{r}(s, a)+\sqrt{\frac{5}{8} T \log \left(\frac{8 T}{\delta}\right)}
$$
が確率$1-\frac{\delta}{12 T^{5 / 4}}$以上で成立します．
後は書き直せば終わりです．

---



期待リグレットを出す場合は真のMDPが$\mathfrak{M}$に入っていない場合のリグレットを出す必要がありますが，面倒なので省略します．

### $M \in \mathfrak{M}_k$の場合のリグレット　

Optimismは後で示します．
先にOptimismが成立している場合のリグレットを出してしまいましょう．
まず，Extended Value Iterationの仕組みから，

$$
\tilde{\rho}_k \geq \rho^*-\frac{1}{\sqrt{t_k}}
$$

が成立します．よって，
$$
\Delta_k \leq \sum_{s, a} v_k(s, a)\left(\rho^*-\bar{r}(s, a)\right) \leq \sum_{s, a} v_k(s, a)\left(\tilde{\rho}_k-\bar{r}(s, a)\right)+\sum_{s, a} \frac{v_k(s, a)}{\sqrt{t_k}}
$$

が成り立ちます．
